In [1]:
%config IPCompleter.greedy=True
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})

{'CodeCell': {'cm_config': {'autoCloseBrackets': False}},
 'Cell': {'cm_config': {'lineNumbers': True}}}

In [2]:
from plnn.mnist_sequential import Net
import torch
import torch.nn as nn
from torchvision.datasets.mnist import MNIST
import torch.utils.data
import numpy as np
import torch.nn.functional as F
from torchvision import datasets, transforms

In [3]:
def generate_domain(input_tensor, eps_size):
    return torch.stack((input_tensor - eps_size, input_tensor + eps_size))

In [4]:
model = Net()
model.load_state_dict(torch.load('save/mnist_sequential_cnn.pt'))
model.cuda()
dataset = MNIST('./data', train=True, download=True,
                transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
                ])),  # load the testing dataset
batch_size=10
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=False, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(dataset, batch_size=1)#retrieve items 1 at a time
seed = 0

In [5]:
class VerificationNetwork(nn.Module):
    def __init__(self, in_features, out_features, base_network, out_function, true_class_index):
        super(VerificationNetwork, self).__init__()
        self.true_class_index = true_class_index
        self.out_function = out_function
        self.base_network = base_network
        self.out_features = out_features
        self.in_features = in_features
        self.property_layer = self.attach_property_layers(self.base_network, self.true_class_index)

    '''need to  repeat this method for each class so that it describes the distance between the corresponding class 
    and the closest other class'''
    def attach_property_layers(self, model: Net, true_class_index: int):
        n_classes = model.fc2.out_features
        cases = []
        for i in range(n_classes):
            if i == true_class_index:
                continue
            case = [0] * n_classes  # list of zeroes
            case[true_class_index] = 1  # sets the property to 1
            case[i] = -1
            cases.append(case)
        weights = np.array(cases)
        weightTensor = nn.Linear(in_features=n_classes, out_features=n_classes,
                                 bias=False)
        weightTensor.weight.data = torch.from_numpy(weights).float()
        return weightTensor

    def forward(self, x):
        x = self.base_network(x)
        x = self.property_layer(x)
        print(x)
        print(x.size())
        return torch.min(x,dim=1,keepdim=True)


In [6]:
#get the data and label
data, target =next(iter(test_loader))
print(f'data size:{data.size()}')
# print(data[0])
# create the domain
domain_raw = generate_domain(data,0.001)
data_size=data.size()
print(f'domain size:{domain_raw.size()}')


data size:torch.Size([10, 1, 28, 28])
domain size:torch.Size([2, 10, 1, 28, 28])


In [7]:
test_out=model(data.cuda())
print(test_out.size())

torch.Size([10, 10])


In [8]:
domain=domain_raw.view(2,batch_size,-1)
print(domain.size())

torch.Size([2, 10, 784])


In [9]:
# global_ub_point, global_ub = net.get_upper_bound(domain)
# global_lb = net.get_lower_bound(domain)


def get_upper_bound(domain):
    #we try get_upper_bound
    nb_samples = 1024
    nb_inp = domain.size()[2:]  #get last dimensions
    print(nb_inp)
    # Not a great way of sampling but this will be good enough
    # We want to get rows that are >= 0
    rand_samples_size = [batch_size, nb_samples] + list(nb_inp)
    print(rand_samples_size)
    rand_samples = torch.zeros(rand_samples_size)
    print(rand_samples.size())
    # print(rand_samples)
    rand_samples.uniform_(0, 1)
    # print(rand_samples)
    print(rand_samples.size())
    domain_lb = domain.select(0, 0).contiguous()
    domain_ub = domain.select(0, 1).contiguous()
    # print(domain_lb)
    print(domain_lb.size())
    # print(domain_ub)
    print(domain_ub.size())
    domain_width = domain_ub - domain_lb
    print(domain_width.size())
    print(domain_lb.view([batch_size, 1] + list(nb_inp)).size())
    print(domain_width.view([batch_size, 1] + list(nb_inp)).size())
    domain_lb = domain_lb.view([batch_size, 1] + list(nb_inp)).expand(
        [batch_size, nb_samples] + list(nb_inp))  #expand the initial point for the number of examples
    print(domain_lb.size())
    domain_width = domain_width.view([batch_size, 1] + list(nb_inp)).expand(
        [batch_size, nb_samples] + list(nb_inp))  #expand the width for the number of examples
    print(domain_width.size())
    #those should be the same
    print(domain_width.size())
    print(rand_samples.size())
    inps = domain_lb + domain_width * rand_samples
    # print(inps) #each row shuld be different
    print(inps.size())
    #now flatten the first dimension into the second
    flattened_size = [inps.size(0) * inps.size(1)] + list(inps.size()[2:])
    print(flattened_size)
    #rearrange the tensor so that is consumable by the model
    print(data_size)
    examples_data_size = [flattened_size[0]] + list(data_size[1:])  #the expected dimension of the example tensor
    print(examples_data_size)
    var_inps = torch.Tensor(inps).view(examples_data_size)
    print(var_inps.size())  #should match data_size
    print(inps.size())
    # print(inps[0][0])
    # print(inps[0][1])
    # print(var_inps[0][0])
    # print(var_inps[1][0])
    outs = model.forward(var_inps.cuda())  #gets the input for the values
    print(outs.size())
    print(outs[0])  #those two should be very similar but different because they belong to two different random examples
    print(outs[1])
    print(target.unsqueeze(1))
    target_expanded = target.unsqueeze(1).expand(
        [batch_size, nb_samples])  #generates nb_samples copies of the target vector, all rows should be the same
    print(target_expanded.size())
    print(target_expanded)
    target_idxs = target_expanded.contiguous().view(
        batch_size * nb_samples)  #contains a list of indices that tells which columns out of the 10 classes to pick
    print(target_idxs.size())  #the first dimension should match
    print(outs.size())
    print(outs[target_idxs[0]].size())
    outs_true_class = outs.gather(1, target_idxs.cuda().view(-1,
                                                             1))  #we choose dimension 1 because it's the one we want to reduce
    print(outs_true_class.size())
    # print(outs[0])
    # print(target_idxs[1])
    # print(outs[1][0])#these two should be similar but different because they belong to different examples
    # print(outs[0][0])
    print(outs_true_class.size())
    outs_true_class_resized = outs_true_class.view(batch_size, nb_samples)
    print(outs_true_class_resized.size())  #resize outputs so that they each row is a different element of each batch
    upper_bound, idx = torch.min(outs_true_class_resized,
                                 dim=1)  #this returns the distance of the network output from the given class, it selects the class which is furthest from the current one
    print(upper_bound.size())
    print(idx.size())
    print(idx)
    print(upper_bound)
    # rearranged_idx=idx.view(list(inps.size()[0:2]))
    # print(rearranged_idx.size()) #rearranged idx contains the indexes of the minimum class for each example, for each element of the batch
    print(f'idx size {idx.size()}')
    print(f'inps size {inps.size()}')
    print(idx[0])
    # upper_bound = upper_bound[0]
    unsqueezed_idx = idx.cuda().view(-1, 1)
    print(f'single size {inps[0][unsqueezed_idx[0][0]][:].size()}')
    print(f'single size {inps[1][unsqueezed_idx[1][0]][:].size()}')
    print(f'single size {inps[2][unsqueezed_idx[2][0]][:].size()}')
    ub_point = [inps[x][idx[x]][:].numpy() for x in range(idx.size()[0])]
    ub_point = torch.tensor(ub_point)
    print(
        ub_point)  #ub_point represents the input that amongst all examples returns the minimum response for the appropriate class
    print(ub_point.size())
    # print(unsqueezed_idx.size())
    # ub_point = torch.gather(inps.cuda(),1,unsqueezed_idx.cuda())#todo for some reason it doesn't want to work
    # print(ub_point.size())
    return ub_point, upper_bound


In [10]:
#test the method
get_upper_bound(domain)

torch.Size([784])
[10, 1024, 784]
torch.Size([10, 1024, 784])
torch.Size([10, 1024, 784])
torch.Size([10, 784])
torch.Size([10, 784])
torch.Size([10, 784])
torch.Size([10, 1, 784])
torch.Size([10, 1, 784])
torch.Size([10, 1024, 784])
torch.Size([10, 1024, 784])
torch.Size([10, 1024, 784])
torch.Size([10, 1024, 784])
torch.Size([10, 1024, 784])
[10240, 784]
torch.Size([10, 1, 28, 28])
[10240, 1, 28, 28]
torch.Size([10240, 1, 28, 28])
torch.Size([10, 1024, 784])
torch.Size([10240, 10])
tensor([-1.1007e+01, -1.5360e+01, -8.2825e+00, -6.8684e+00, -1.4874e+01,
        -1.0945e+01, -2.1491e+01, -1.4362e-03, -1.0061e+01, -9.6479e+00],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([-1.1009e+01, -1.5363e+01, -8.2847e+00, -6.8711e+00, -1.4878e+01,
        -1.0948e+01, -2.1496e+01, -1.4324e-03, -1.0064e+01, -9.6508e+00],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[7],
        [2],
        [1],
        [0],
        [4],
        [1],
        [4],
        [9],
        [5],


(tensor([[-0.4246, -0.4235, -0.4234,  ..., -0.4250, -0.4247, -0.4237],
         [-0.4241, -0.4232, -0.4243,  ..., -0.4251, -0.4234, -0.4244],
         [-0.4242, -0.4243, -0.4252,  ..., -0.4250, -0.4244, -0.4245],
         ...,
         [-0.4240, -0.4235, -0.4251,  ..., -0.4247, -0.4247, -0.4238],
         [-0.4235, -0.4233, -0.4240,  ..., -0.4239, -0.4240, -0.4239],
         [-0.4235, -0.4236, -0.4236,  ..., -0.4237, -0.4235, -0.4252]]),
 tensor([-1.4400e-03, -2.5005e-03, -9.0146e-03, -5.1594e-04, -1.0918e-02,
         -3.6526e-03, -6.2752e-03, -1.9007e-03, -6.9505e-01, -7.2823e-03],
        device='cuda:0', grad_fn=<MinBackward0>))

In [11]:
#now try to do the lower bound
import gurobipy as grb

In [12]:
'''
input_domain: Tensor containing in each row the lower and upper bound
              for the corresponding dimension
'''
lower_bounds = []
upper_bounds = []
gurobi_vars = []
# These three are nested lists. Each of their elements will itself be a
# list of the neurons after a layer.

gurobi_model = grb.Model()
gurobi_model.setParam('OutputFlag', False)
gurobi_model.setParam('Threads', 1)

Academic license - for non-commercial use only


In [13]:
input_domain=domain.select(1,0)#we use a single domain, not ready for parallelisation yet
print(input_domain.size())

torch.Size([2, 784])


In [14]:
## Do the input layer, which is a special case
inp_lb = []
inp_ub = []
inp_gurobi_vars = []
for dim in range(input_domain.size()[1]):
    ub=input_domain[0][dim]
    lb=input_domain[1][dim]
    v = gurobi_model.addVar(lb=lb, ub=ub, obj=0,
                          vtype=grb.GRB.CONTINUOUS,
                          name=f'inp_{dim}')
    inp_gurobi_vars.append(v)
    inp_lb.append(lb)
    inp_ub.append(ub)
gurobi_model.update()

In [15]:
lower_bounds.append(inp_lb)
upper_bounds.append(inp_ub)
gurobi_vars.append(inp_gurobi_vars)

In [16]:
#now i need to list each layer in the model
layers = []


In [33]:


## Do the other layers, computing for each of the neuron, its upper
## bound and lower bound
layer_idx = 1
# for layer in model.layers:
layer = model.layers[0]
print( type(layer))
new_layer_lb = []
new_layer_ub = []
new_layer_gurobi_vars = []
if type(layer) is nn.Linear:
    for neuron_idx in range(layer.weight.size(0)):
        ub = layer.bias.data[neuron_idx]
        lb = layer.bias.data[neuron_idx]
        lin_expr = layer.bias.data[neuron_idx].item()
        for prev_neuron_idx in range(layer.weight.size(1)):
            coeff = layer.weight.data[neuron_idx, prev_neuron_idx]
            if coeff >= 0:
                ub += coeff*upper_bounds[-1][prev_neuron_idx]
                lb += coeff*lower_bounds[-1][prev_neuron_idx]
            else:
                ub += coeff*lower_bounds[-1][prev_neuron_idx]
                lb += coeff*upper_bounds[-1][prev_neuron_idx]
            lin_expr += coeff.item() * gurobi_vars[-1][prev_neuron_idx]
        v = gurobi_model.addVar(lb=lb, ub=ub, obj=0,
                              vtype=grb.GRB.CONTINUOUS,
                              name=f'lay{layer_idx}_{neuron_idx}')
        gurobi_model.addConstr(v == lin_expr)
        gurobi_model.update()

        gurobi_model.setObjective(v, grb.GRB.MINIMIZE)
        gurobi_model.optimize()
        assert gurobi_model.status == 2, "LP wasn't optimally solved"
        # We have computed a lower bound
        lb = v.X
        v.lb = lb

        # Let's now compute an upper bound
        gurobi_model.setObjective(v, grb.GRB.MAXIMIZE)
        gurobi_model.update()
        gurobi_model.reset()
        gurobi_model.optimize()
        assert gurobi_model.status == 2, "LP wasn't optimally solved"
        ub = v.X
        v.ub = ub

        new_layer_lb.append(lb)
        new_layer_ub.append(ub)
        new_layer_gurobi_vars.append(v)
elif type(layer) == nn.ReLU:
    for neuron_idx, pre_var in enumerate(gurobi_vars[-1]):
        pre_lb = lower_bounds[-1][neuron_idx]
        pre_ub = upper_bounds[-1][neuron_idx]

        v = gurobi_model.addVar(lb=max(0, pre_lb),
                              ub=max(0, pre_ub),
                              obj=0,
                              vtype=grb.GRB.CONTINUOUS,
                              name=f'ReLU{layer_idx}_{neuron_idx}')
        if pre_lb >= 0 and pre_ub >= 0:
            # The ReLU is always passing
            gurobi_model.addConstr(v == pre_var)
            lb = pre_lb
            ub = pre_ub
        elif pre_lb <= 0 and pre_ub <= 0:
            lb = 0
            ub = 0
            # No need to add an additional constraint that v==0
            # because this will be covered by the bounds we set on
            # the value of v.
        else:
            lb = 0
            ub = pre_ub
            gurobi_model.addConstr(v >= pre_var)

            slope = pre_ub / (pre_ub - pre_lb)
            bias = - pre_lb * slope
            gurobi_model.addConstr(v <= slope * pre_var + bias)

        new_layer_lb.append(lb)
        new_layer_ub.append(ub)
        new_layer_gurobi_vars.append(v)
elif type(layer) == nn.MaxPool1d:
    assert layer.padding == 0, "Non supported Maxpool option"
    assert layer.dilation == 1, "Non supported MaxPool option"
    nb_pre = len(gurobi_vars[-1])
    window_size = layer.kernel_size
    stride = layer.stride

    pre_start_idx = 0
    pre_window_end = pre_start_idx + window_size

    while pre_window_end <= nb_pre:
        lb = max(lower_bounds[-1][pre_start_idx:pre_window_end])
        ub = max(upper_bounds[-1][pre_start_idx:pre_window_end])

        neuron_idx = pre_start_idx // stride

        v = gurobi_model.addVar(lb=lb, ub=ub, obj=0, vtype=grb.GRB.CONTINUOUS,
                              name=f'Maxpool{layer_idx}_{neuron_idx}')
        all_pre_var = 0
        for pre_var in gurobi_vars[-1][pre_start_idx:pre_window_end]:
            gurobi_model.addConstr(v >= pre_var)
            all_pre_var += pre_var
        all_lb = sum(lower_bounds[-1][pre_start_idx:pre_window_end])
        max_pre_lb = lb
        gurobi_model.addConstr(all_pre_var >= v + all_lb - max_pre_lb)

        pre_start_idx += stride
        pre_window_end = pre_start_idx + window_size

        new_layer_lb.append(lb)
        new_layer_ub.append(ub)
        new_layer_gurobi_vars.append(v)
elif type(layer) == View:
#     continue
    print(type(layer))
else:
    raise NotImplementedError

lower_bounds.append(new_layer_lb)
upper_bounds.append(new_layer_ub)
gurobi_vars.append(new_layer_gurobi_vars)

layer_idx += 1

# Assert that this is as expected a network with a single output
assert len(gurobi_vars[-1]) == 1, "Network doesn't have scalar output"

gurobi_model.update()

<class 'torch.nn.modules.linear.Linear'>
-0.15002122521400452
tensor(0.0158, device='cuda:0')
<gurobi.Var inp_0>
0.015848033130168915
tensor(-0.0183, device='cuda:0')
<gurobi.Var inp_1>
-0.018317729234695435
tensor(-0.0095, device='cuda:0')
<gurobi.Var inp_2>
-0.009477861225605011
tensor(0.0142, device='cuda:0')
<gurobi.Var inp_3>
0.014208938926458359
tensor(-0.0362, device='cuda:0')
<gurobi.Var inp_4>
-0.036177001893520355
tensor(0.0189, device='cuda:0')
<gurobi.Var inp_5>
0.018864255398511887
tensor(-0.0099, device='cuda:0')
<gurobi.Var inp_6>
-0.009901568293571472
tensor(0.0156, device='cuda:0')
<gurobi.Var inp_7>
0.015615233220160007
tensor(0.0024, device='cuda:0')
<gurobi.Var inp_8>
0.0024106528144329786
tensor(-0.0069, device='cuda:0')
<gurobi.Var inp_9>
-0.006927243433892727
tensor(0.0074, device='cuda:0')
<gurobi.Var inp_10>
0.007351519539952278
tensor(-0.0008, device='cuda:0')
<gurobi.Var inp_11>
-0.0007923822267912328
tensor(0.0105, device='cuda:0')
<gurobi.Var inp_12>
0.0104

<gurobi.Var inp_233>
0.010337105952203274
tensor(0.0086, device='cuda:0')
<gurobi.Var inp_234>
0.008644483052194118
tensor(0.0310, device='cuda:0')
<gurobi.Var inp_235>
0.030967069789767265
tensor(-0.0032, device='cuda:0')
<gurobi.Var inp_236>
-0.003154176287353039
tensor(0.0359, device='cuda:0')
<gurobi.Var inp_237>
0.03588251769542694
tensor(0.0466, device='cuda:0')
<gurobi.Var inp_238>
0.04663754999637604
tensor(-0.0060, device='cuda:0')
<gurobi.Var inp_239>
-0.0059913285076618195
tensor(-0.0031, device='cuda:0')
<gurobi.Var inp_240>
-0.0030508602503687143
tensor(0.0098, device='cuda:0')
<gurobi.Var inp_241>
0.00978060346096754
tensor(0.0308, device='cuda:0')
<gurobi.Var inp_242>
0.03083907812833786
tensor(0.0048, device='cuda:0')
<gurobi.Var inp_243>
0.004824853036552668
tensor(-0.0100, device='cuda:0')
<gurobi.Var inp_244>
-0.01001178938895464
tensor(0.0020, device='cuda:0')
<gurobi.Var inp_245>
0.002016883809119463
tensor(-0.0296, device='cuda:0')
<gurobi.Var inp_246>
-0.02962655

<gurobi.Var inp_476>
-0.036758072674274445
tensor(0.0094, device='cuda:0')
<gurobi.Var inp_477>
0.009370174258947372
tensor(0.0100, device='cuda:0')
<gurobi.Var inp_478>
0.010025237686932087
tensor(-0.0300, device='cuda:0')
<gurobi.Var inp_479>
-0.029962627217173576
tensor(0.0155, device='cuda:0')
<gurobi.Var inp_480>
0.015523381531238556
tensor(-0.0145, device='cuda:0')
<gurobi.Var inp_481>
-0.014483821578323841
tensor(0.0039, device='cuda:0')
<gurobi.Var inp_482>
0.003907417878508568
tensor(-0.0028, device='cuda:0')
<gurobi.Var inp_483>
-0.002773281652480364
tensor(0.0170, device='cuda:0')
<gurobi.Var inp_484>
0.01698952168226242
tensor(-0.0231, device='cuda:0')
<gurobi.Var inp_485>
-0.023055752739310265
tensor(-0.0271, device='cuda:0')
<gurobi.Var inp_486>
-0.027076473459601402
tensor(0.0429, device='cuda:0')
<gurobi.Var inp_487>
0.04289839416742325
tensor(0.0217, device='cuda:0')
<gurobi.Var inp_488>
0.021716387942433357
tensor(0.0059, device='cuda:0')
<gurobi.Var inp_489>
0.005867

tensor(0.0063, device='cuda:0')
<gurobi.Var inp_738>
0.006285423878580332
tensor(-0.0209, device='cuda:0')
<gurobi.Var inp_739>
-0.02089674584567547
tensor(-0.0377, device='cuda:0')
<gurobi.Var inp_740>
-0.03769154101610184
tensor(-0.0305, device='cuda:0')
<gurobi.Var inp_741>
-0.030524343252182007
tensor(0.0081, device='cuda:0')
<gurobi.Var inp_742>
0.008071579970419407
tensor(0.0111, device='cuda:0')
<gurobi.Var inp_743>
0.011118737980723381
tensor(-0.0339, device='cuda:0')
<gurobi.Var inp_744>
-0.033904433250427246
tensor(0.0160, device='cuda:0')
<gurobi.Var inp_745>
0.016039980575442314
tensor(0.0243, device='cuda:0')
<gurobi.Var inp_746>
0.024275001138448715
tensor(0.0260, device='cuda:0')
<gurobi.Var inp_747>
0.02601543813943863
tensor(0.0170, device='cuda:0')
<gurobi.Var inp_748>
0.01701454631984234
tensor(-0.0091, device='cuda:0')
<gurobi.Var inp_749>
-0.00914730504155159
tensor(0.0211, device='cuda:0')
<gurobi.Var inp_750>
0.021053964272141457
tensor(0.0253, device='cuda:0')
<

AssertionError: LP wasn't optimally solved

In [ ]:
print(data.size())
true_class=torch.argmax(model(data.cuda()),dim=1)
print(true_class.size())
print(true_class)
single_true_class=true_class[0].item()
print(single_true_class)
verification_model=VerificationNetwork(28,10,model,model.forward,single_true_class)
verification_model.cuda()

result=verification_model.forward(data.cuda())
print(result)